In [1]:
import numpy as np
import pandas as pd
import sklearn.model_selection as model_selection
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf

import tensorflow.keras as keras
from keras.preprocessing.image import ImageDataGenerator
import itertools

import pydot
import matplotlib.pyplot as plt

In [2]:
# Test for GPU and CUDA

print('Devices List: ', tf.config.list_physical_devices('GPU'))
print('Is built with CUDA:', tf.test.is_built_with_cuda())

Devices List:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is built with CUDA: True


In [3]:
# List, dictionaries of class number and labels.

label_dict = {0: 'Nucleoplasm', 1: 'Nuclear membrane', 2: 'Nucleoli', 3: 'Nucleoli fibrillar center', 4: 'Nuclear speckles', 5: 'Nuclear bodies', 6: 'Endoplasmic reticulum',
         7: 'Golgi apparatus', 8: 'Peroxisomes', 9: 'Endosomes', 10: 'Lysosomes', 11: 'Intermediate filaments', 12: 'Actin filaments', 13: 'Focal adhesion sites', 
         14: 'Microtubules', 15: 'Microtubule ends', 16: 'Cytokinetic bridge', 17: 'Mitotic spindle', 18: 'Microtubule organizing center', 19: 'Centrosome', 20: 'Lipid droplets',
         21: 'Plasma membrane', 22: 'Cell junctions', 23: 'Mitochondria', 24: 'Aggresome', 25: 'Cytosol', 26: 'Cytoplasmic bodies', 27: 'Rods & rings'}

label_list = ['Nucleoplasm', 'Nuclear membrane', 'Nucleoli', 'Nucleoli fibrillar center', 'Nuclear speckles', 'Nuclear bodies', 'Endoplasmic reticulum', 
              'Golgi apparatus', 'Peroxisomes', 'Endosomes', 'Lysosomes', 'Intermediate filaments', 'Actin filaments', 'Focal adhesion sites', 
              'Microtubules', 'Microtubule ends', 'Cytokinetic bridge', 'Mitotic spindle', 'Microtubule organizing center', 'Centrosome', 'Lipid droplets', 
              'Plasma membrane', 'Cell junctions', 'Mitochondria', 'Aggresome', 'Cytosol', 'Cytoplasmic bodies', 'Rods & rings']

map_label_number = {'Nucleoplasm': 0, 'Nuclear membrane': 1, 'Nucleoli': 2, 'Nucleoli fibrillar center': 3, 'Nuclear speckles': 4, 'Nuclear bodies': 5, 'Endoplasmic reticulum': 6, 
                    'Golgi apparatus': 7, 'Peroxisomes': 8, 'Endosomes': 9, 'Lysosomes': 10, 'Intermediate filaments': 11, 'Actin filaments': 12, 'Focal adhesion sites': 13, 
                    'Microtubules': 14, 'Microtubule ends': 15, 'Cytokinetic bridge': 16, 'Mitotic spindle': 17, 'Microtubule organizing center': 18, 'Centrosome': 19, 'Lipid droplets': 20, 
                    'Plasma membrane': 21, 'Cell junctions': 22, 'Mitochondria': 23, 'Aggresome': 24, 'Cytosol': 25, 'Cytoplasmic bodies': 26, 'Rods & rings': 27}

In [4]:
# Set the path to train set

path_to_train = './train/'

In [5]:
# Global Variables

NUMBER_OF_LABELS = 28

NUMBER_OF_CLASSES = 27

INPUT_SHAPE = (512, 512, 3) # Images all have the same dimension
BATCH_SIZE = 1 # batch size for model learning. 8 gives OOM

STEPS_PER_EPOCH = 150
VALIDATION_STEPS = 3000
NUM_EPOCH = 40

In [6]:
ground_truth = pd.read_csv('train.csv')
ground_truth['List Target'] = ground_truth['Target'].apply(lambda string: sorted(list(map(int, string.split(' ')))))
ground_truth['Label Target'] =  ground_truth['List Target'].apply(lambda lis: [label_list[idx] for idx in lis])
ground_truth['Label'] =  ground_truth['Label Target'].apply(lambda lis: ', '.join(lis))
ground_truth['Path'] = path_to_train + ground_truth['Id']

In [7]:
# Get train set and validation set

train, val = model_selection.train_test_split(ground_truth, test_size = 0.2, random_state = 42)

In [8]:
# OneHotEncoding

def get_clean_data(df):
    targets = []
    paths = []
    for _, row in df.iterrows():
        target_np = np.zeros(len(label_list))
        t = [int(t) for t in row.Target.split()]
        target_np[t] = 1
        targets.append(target_np)
        paths.append(row.Path)
    return np.array(paths), np.array(targets)


def load_data(path, target, channels = 3):
    if channels == 3:
        red = tf.squeeze(tf.image.decode_png(tf.io.read_file(path + '_red.png'), channels = 1), [2])
        blue = tf.squeeze(tf.image.decode_png(tf.io.read_file(path + '_blue.png'), channels = 1), [2])
        green = tf.squeeze(tf.image.decode_png(tf.io.read_file(path + '_green.png'), channels = 1), [2])
        img = tf.stack((red, green, blue), axis=2)
    
    elif channels == 4:
        red = tf.squeeze(tf.image.decode_png(tf.io.read_file(path + '_red.png'), channels = 1), [2])
        blue = tf.squeeze(tf.image.decode_png(tf.io.read_file(path + '_blue.png'), channels = 1), [2])
        green = tf.squeeze(tf.image.decode_png(tf.io.read_file(path + '_green.png'), channels = 1), [2])
        yellow = tf.squeeze(tf.image.decode_png(tf.io.read_file(path + '_yellow.png'), channels = 1), [2])
        img = tf.stack((red, green, blue, yellow), axis=2)
        
    return img, target


AUTOTUNE = tf.data.experimental.AUTOTUNE

In [9]:
train_path, train_target = get_clean_data(train)
val_path, val_target = get_clean_data(val)

#print(f'Train path shape: {train_path.shape}')
#print(f'Train target shape: {train_target.shape}')
#print(f'Val path shape: {val_path.shape}')
#print(f'Val target shape: {val_target.shape}')

In [10]:
#train_data = tf.data.Dataset.from_tensor_slices((train_path, train_target))
#val_data = tf.data.Dataset.from_tensor_slices((val_path, val_target))

In [11]:
#train_data = train_data.map(load_data, num_parallel_calls = AUTOTUNE) # prendiamo ogni elemento di train_data (che sono path) e li mappiamo nella rispettiva immagine
#val_data = val_data.map(load_data, num_parallel_calls = AUTOTUNE) # <ParallelMapDataset shapes: ((None, None, 3), (28,)), types: (tf.uint8, tf.float64)>

In [12]:
#train_data_batches = train_data.batch(BATCH_SIZE).prefetch(buffer_size = AUTOTUNE)
#val_data_batches = val_data.batch(BATCH_SIZE).prefetch(buffer_size = AUTOTUNE) # <PrefetchDataset shapes: ((None, None, None, 3), (None, 28)), types: (tf.uint8, tf.float64)>

In [13]:
############################################

In [14]:
def input_generator(path_target_generator, batch_size = 31072):
#def input_generator(path_target_generator):
    dt = np.float32
    i = 0
    while i < batch_size:
        path, target = next(path_target_generator)
        img, _ = load_data(path, target) 
        img = np.array(img).astype(dt)

        #inputs  = (img, 
        #           img 
        #           )

        yield (img, img), target
        i += 1

In [15]:
train_path_target_generator = itertools.cycle(zip(train_path, train_target))
val_path_target_generator = itertools.cycle(zip(val_path, val_target))

In [16]:
output_signature = (
                    (tf.TensorSpec(shape=INPUT_SHAPE, dtype=tf.float32), tf.TensorSpec(shape=INPUT_SHAPE, dtype=tf.float32)),
                    tf.TensorSpec(shape=(NUMBER_OF_LABELS), dtype=tf.float32)
                    )
types = ( (tf.float32,tf.float32),
          (tf.float32) )

multiple_input_train = tf.data.Dataset.from_generator(lambda: input_generator(train_path_target_generator, len(train_path)), output_types = types)
multiple_input_val = tf.data.Dataset.from_generator(lambda: input_generator(val_path_target_generator, len(val_path)), output_types = types)

In [17]:
multiple_input_train_batches = multiple_input_train.batch(BATCH_SIZE).prefetch(buffer_size = AUTOTUNE)
multiple_input_val_batches = multiple_input_val.batch(BATCH_SIZE).prefetch(buffer_size = AUTOTUNE)

In [18]:
###########################################

In [19]:
# ResNet50 Workflow

resnet_model = keras.applications.ResNet50(include_top = False, weights = 'imagenet') # include_top = False dovrebbe permettermi di personalizzare l'input layer
resnet_model.trainable = True

res_input = keras.layers.Input(shape = INPUT_SHAPE) # , name = 'Load RGB Images SX'
res = resnet_model(res_input)
res = keras.layers.Flatten()(res) # create a single array
res = keras.layers.Dropout(0.5)(res) # reduce overfitting
res = keras.layers.Dense(256, activation = 'relu')(res)
res = keras.layers.Dropout(0.5)(res)
res_output = keras.layers.Dense(28, activation = 'sigmoid')(res) # 28, one for each class # , name = 'Label_Weights'
res_output = keras.layers.Reshape((1,28))(keras.layers.Dense(28, activation = 'sigmoid')(res))
#resnet_model = keras.models.Model(res_input, res_output)

In [20]:
# Xception Workflow

xception_model = keras.applications.Xception(include_top = False, weights = 'imagenet') # include_top = False dovrebbe permettermi di personalizzare l'input layer
xception_model.trainable = True

xception_input = keras.layers.Input(shape = INPUT_SHAPE) # , name = 'Load RGB Images DX'
xce = xception_model(xception_input)
xce = keras.layers.Flatten()(xce) # create a single array
xce = keras.layers.Dropout(0.5)(xce) # reduce overfitting
xce = keras.layers.Dense(256, activation = 'relu')(xce)
xce = keras.layers.Dropout(0.5)(xce)
xce_output = keras.layers.Dense(27, activation = 'sigmoid')(xce) # 28, one for each class # , name = 'Cell_Type_Weights'
xce_output = keras.layers.Reshape((27,1))(keras.layers.Dense(27, activation = 'sigmoid')(xce))
#xception_model = keras.models.Model(xception_input, xce_output)

In [21]:
# Merge Layer using Dot Product
dot_merge = keras.layers.Dot(axes=[1, 2], name = 'Dot_Product')([res_output, xce_output])
dot_merge = keras.layers.Flatten()(dot_merge)
dot_merge = keras.layers.Dense(378, activation = 'relu')(dot_merge)
dot_merge = keras.layers.Dropout(0.5)(dot_merge)
hpav100_output = keras.layers.Dense(28, activation = 'sigmoid')(dot_merge) # , name = 'HPAV100_Output'


In [22]:
# Full HPAV100 Model

hpav100_model = keras.models.Model(inputs = [res_input, xception_input], outputs = hpav100_output)

In [23]:
hpav100_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
resnet50 (Functional)           (None, None, None, 2 23587712    input_2[0][0]                    
__________________________________________________________________________________________________
xception (Functional)           (None, None, None, 2 20861480    input_4[0][0]                    
______________________________________________________________________________________________

In [24]:
keras.utils.plot_model(hpav100_model, "hpav100_model.png", show_shapes = True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [25]:
hpav100_model.compile(optimizer = keras.optimizers.Adam(1e-4), loss = 'binary_crossentropy', metrics = 'binary_accuracy')

In [ ]:
# Uncomment and run for training the model

history = hpav100_model.fit(multiple_input_train_batches,
                            steps_per_epoch = STEPS_PER_EPOCH,
                            validation_data = multiple_input_val_batches,
                            validation_steps = VALIDATION_STEPS,
                            epochs = NUM_EPOCH)

Epoch 1/40
150/150 [==============================] - 208s 1s/step - loss: 0.4761 - binary_accuracy: 0.7831 - val_loss: 0.1870 - val_binary_accuracy: 0.9425
Epoch 2/40
150/150 [==============================] - 187s 1s/step - loss: 0.2326 - binary_accuracy: 0.9295 - val_loss: 0.1849 - val_binary_accuracy: 0.9352
Epoch 3/40
150/150 [==============================] - 183s 1s/step - loss: 0.2147 - binary_accuracy: 0.9299 - val_loss: 0.1846 - val_binary_accuracy: 0.9422
Epoch 4/40
150/150 [==============================] - 183s 1s/step - loss: 0.2161 - binary_accuracy: 0.9338 - val_loss: 0.1931 - val_binary_accuracy: 0.9415
Epoch 5/40
150/150 [==============================] - 182s 1s/step - loss: 0.2291 - binary_accuracy: 0.9342 - val_loss: 0.1786 - val_binary_accuracy: 0.9422
Epoch 6/40
150/150 [==============================] - 183s 1s/step - loss: 0.2155 - binary_accuracy: 0.9348 - val_loss: 0.1883 - val_binary_accuracy: 0.9415
Epoch 7/40
150/150 [==============================] - 183s

In [ ]:
# Plot History

pd.DataFrame(history.history).plot(figsize=(12,4))
plt.grid(True)

In [ ]:
hpav100_model.save('./hpa_HPAV100_model.tf')

In [ ]:
###### Predict model on validation to get threshold for prediction over test set

In [ ]:
def get_frequency_table(ground_truth):
    all_label = pd.DataFrame(ground_truth['Label Target'].sum(), columns = ['Label']).copy()
    frequency_table = all_label['Label'].value_counts().reset_index().copy().rename(columns = {'index': 'Label', 'Label': 'Frequency'})
    frequency_table['Numeric Label'] = pd.DataFrame(frequency_table['Label']).replace({'Label': map_label_number}).copy()
    frequency_table['Relative Frequency'] = frequency_table['Frequency'] / frequency_table['Frequency'].sum()
    frequency_table['Relative Frequency %'] = (frequency_table['Frequency'] / frequency_table['Frequency'].sum()).apply(lambda number: '{:.2%}'.format(number))
    frequency_table[['Numeric Label', 'Label', 'Frequency', 'Relative Frequency', 'Relative Frequency %']].set_index('Numeric Label')
    return frequency_table

def get_threshold_from_validation(model, val_data_batches, ground_truth, batch_size = 4, verbose = 1, steps = None):
    frequency_table = get_frequency_table(ground_truth)
    prediction = model.predict(val_data_batches, batch_size = BATCH_SIZE, verbose = verbose, steps = steps)
    freq = list(frequency_table[['Numeric Label', 'Relative Frequency']].sort_values(by='Numeric Label')['Relative Frequency'])
    thresh = np.diagonal(np.array(pd.DataFrame(prediction).quantile([1 - f for f in freq])))
    return thresh

In [ ]:
# Load the model
model_HPAV100 = keras.models.load_model('hpa_HPAV100_model.tf')

In [ ]:
# Get threshold for prediction
thresh = get_threshold_from_validation(model_HPAV100, multiple_input_val_batches, ground_truth, batch_size = BATCH_SIZE, verbose = 1, steps = None)
thresh